# Example 0: Convert external data

In this example we will download external data that will be used to estimate relative moment tensors for seismic aftershocks related to the 2015-2017 Pamir earthquake sequence and convert them to the data formats required by *relMT*. For background information on the data set, please refer to [Bloch et al. (2023, GJI)](https://doi.org/10.1093/gji/ggac473).

> **Note**
>
> This turtorial relies on the commandline tools `cd` `mkdir` and `wget`.
>
> They are part of most operating systems, but not all.
>
> `head` and `ls` are also used, but not required.

We will start by importing modules required for this turtorial.

In [20]:

from relmt import io, extra, mt, utils

## Download external data

We create a folder `ext/` to store the external data. For this example, we access the data publications

Bloch et al. (2022) Earthquake and Moment Tensor Catalogs of the 2015-2017 Pamir Earthquake Sequence. *GFZ Data Services*. https://doi.org/10.5880/fidgeo.2022.007

and

Bloch et al. (2026): Earthquake Phase Arrival Times of the 2015-2017 Pamir Earthquake Sequence. GFZ Data Services. https://doi.org/10.5880/fidgeo.2026.006

Let us create the directory, change into it and download the data.

In [21]:
# Make the directory, no warning if exists
! mkdir -p ext data

# -nc avoids re-downloading, -nv is less verbose, -P sets target directory
! wget -nc -nv -P ext https://datapub.gfz.de/download/10.5880.FIDGEO.2022.007Geazgf/2022-007_Bloch-et-al_seismic_event_catalog.txt
! wget -nc -nv -P ext https://datapub.gfz.de/download/10.5880.FIDGEO.2022.007Geazgf/2022-007_Bloch-et-al_moment_tensor_catalog.txt
! wget -nc -nv -P ext https://datapub.gfz.de/download/10.5880.FIDGEO.2026.006-Kacenu/2026-006_bloch-et-al_2015-2017_pamir_bulletin.xml


2026-01-27 14:46:38 URL:https://datapub.gfz.de/download/10.5880.FIDGEO.2022.007Geazgf/2022-007_Bloch-et-al_seismic_event_catalog.txt [1885974/1885974] -> "ext/2022-007_Bloch-et-al_seismic_event_catalog.txt" [1]
2026-01-27 14:46:39 URL:https://datapub.gfz.de/download/10.5880.FIDGEO.2022.007Geazgf/2022-007_Bloch-et-al_moment_tensor_catalog.txt [5984/5984] -> "ext/2022-007_Bloch-et-al_moment_tensor_catalog.txt" [1]
2026-01-27 14:46:41 URL:https://datapub.gfz.de/download/10.5880.FIDGEO.2026.006-Kacenu/2026-006_bloch-et-al_2015-2017_pamir_bulletin.xml [143972168/143972168] -> "ext/2026-006_bloch-et-al_2015-2017_pamir_bulletin.xml" [1]


## Create a *relMT* event file

We will now create a *relMT* compliant event file. Let us have a look at the provided catalog file.

In [22]:

! head ext/2022-007_Bloch-et-al_seismic_event_catalog.txt

# License: Creative Commons Attribution 4.0 International License (CC BY 4.0), https://creativecommons.org/licenses/by/4.0/
# Citation: Bloch, W.; Schurr, B.; Yuan, X.; Xu, Q.; Zhao, J.; Murodkulov, S.; Oimuhammadzoda, I. (2022): Earthquake and Moment Tensor Catalogs of the 2015--2017 Pamir Earthquake Sequence. GFZ Data Services. https://doi.org/10.5880/fidgeo.2022.007
#
#Year Month Day Hour Minute Second Timestamp(s)   Longitude(deg) Latitude(deg) Depth(km) P-picks S-picks revised? Method   RMS(s) Magnitude error type Sequence
 2015 08    03  14   37     08.00  1438612628.00  73.95270       38.48570      130.39      9         6   0        simulps  0.280  3.70      0.42  ML   Z       
 2015 08    04  02   10     29.00  1438654229.00  71.71370       37.19950      135.87      8         0   0        simulps  0.270  4.30      0.33  ML   Z       
 2015 08    04  22   19     21.00  1438726761.00  76.81780       39.48380       -3.00      6         0   0        simulps  0.330  2.37      0.33  

We find that the seismic event catalog contains the locations of the events in columns 7, 8 and 9 (0-based indexing), the origin time in column 6 (as a time stamp in seconds since 1970), and the magnitude in cloumn 15. Some of the tools of the `extra` module will help us to:

* Convert the locations from geographic (degree) to UTM (meter) coordinates
* Provide a date string as an event name
* Save it in the `data/` subdirectory for use in the later examples

The Pamir highlands are located in [UTM zone 43 S](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system), in the Tajik Chinese border region.

In [23]:
# The external seismic event catalog file
catf = "ext/2022-007_Bloch-et-al_seismic_event_catalog.txt"

# Geoconverter for UTM zone 43S (Pamir region)
def geoconv(lat, lon, dep):
    return extra.geoconverter_latlon2utm(lat, lon, dep, 43, "S")


# Event name is the origin time in YYYYMMDDHHMMSS format
def nameconv(timetuple):
    yr, mo, da, hr, mn, sc = timetuple
    return f"{yr}{mo}{da}{hr}{mn}{sc[:2]}"


# Read the seismic event catalog into an event dictionary
evd = io.read_ext_event_table(
    catf,
    *(8, 7, 9, 6, 15),  # north, east, depth, magnitude, time
    (0, 1, 2, 3, 4, 5), # year, month, day, hour, minute, second
    geoconv,
    nameconverter=nameconv,
)

# And save it to file for later use
io.write_event_table(evd, "data/events.txt")

print(f"The seismic event catalog has {len(evd)} records.")
print("The first event is:")
print(evd[0])

The seismic event catalog has 11784 records.
The first event is:
Event(north=np.float64(4260226.228515374), east=np.float64(408658.4556633276), depth=np.float64(130389.99999999999), time=1438612628.0, mag=np.float64(3.7), name='20150803143708')


## Creat a reference MT file

Next we will convert the moment tensor file. First, we inspect the file.

In [24]:
! head ext/moment_tensor_catalog_correct_norm.txt

head: cannot open 'ext/moment_tensor_catalog_correct_norm.txt' for reading: No such file or directory


The moment tensor catalog contains the components of the moment tensor. The format follows the Harvard convention used by the [Global CMT project](https://docs.generic-mapping-tools.org/latest/supplements/seis/meca.html) and documented e.g. in [GMT meca -Sm](https://docs.generic-mapping-tools.org/latest/supplements/seis/meca.html). The elements of the moment tensor in [dyne cm] units are decribed in columns 6 to 12 (0-based indexing).

For use in *relMT* we will next
* Convert the event time to a string that matches the event name of the catalog
* Convert from Harvard (R, $\theta$, $\phi$) to North-East-Down coordinates
* Convert from dyne cm to Newton meter (Nm)
* Write the moment tensor table

In [25]:

# The external moment tensor catalog file
mtf = "ext/2022-007_Bloch-et-al_moment_tensor_catalog.txt"

# Event date times in the catalog
evdatetimes = [ev.name for ev in evd.values()]

# Function to convert date and time strings to event names
def nameconverter(datestr, timestr):
    # Remove delimiters from date and time strings and concatenate
    date = datestr.replace("/", "")
    time = timestr.replace(":", "")
    
    # Times do not match exactly, so look up closest event time
    datetime = f"{date}{time}"
    evdatetime = utils.approx_time_lookup([datetime], evdatetimes)

    # Look up in single-element dict
    return evdatetime[datetime]

# Function to convert moment tensors from Harvard format to north-east-down in Nm
def mtconverter(mrr, mtt, mff, mrt, mrf, mtf, exp):
    # Convert to north, east, down ...
    mnn, mee, mdd, mne, mnd, med = mt.rtf2ned(mrr, mtt, mff, mrt, mrf, mtf)

    #  ... and from dyne cm to Nm
    fac = 10**exp * 1e-7
    return (mnn * fac, mee * fac, mdd * fac, mne * fac, mnd * fac, med * fac)

# Read the moment tensor catalog into a moment tensor dictionary
mtd = io.read_ext_mt_table(
    mtf,
    (6, 7, 8, 9, 10, 11),  # mrr, mtt, mff, mrt, mrf, mtf
    (0, 1), # Date strings
    evd,
    mtconverter,  # This mtconverter converts from Harvad to north-east-down convention
    12,  # exponent
    nameconverter,  # The names are used to associate MTs to events
)

# And save it to disk
io.write_mt_table(mtd, "data/reference_mts.txt")

print(f"The MT catalog has {len(mtd)} records, namely:")
print(", ".join(map(str, mtd.keys())))

The MT catalog has 33 records, namely:
498, 515, 877, 972, 2050, 2190, 2695, 4181, 4189, 4273, 4370, 4414, 4472, 4569, 4664, 4839, 5037, 5508, 5883, 5888, 6101, 6102, 7440, 7508, 7640, 8919, 9617, 10715, 10847, 11447, 11481, 11551, 11685


## Conclusion
Let's have a look at the catalog and reference moment tensor file we just created. They now obey the *relMT* conventions, that is:
* units are in meter
* the axes point north, east and down, and
* a global event index (first column) is used to refer to the events

In [26]:
! head data/events.txt

#Number     Northing      Easting       Depth         Origintime Magnitude                 Name
# (int)      (meter)      (meter)     (meter)          (seconds)       (-)                (str)
      0  4260226.229   408658.456  130390.000  1438612628.000000    3.7000       20150803143708
      1  4122064.455   208320.271  135870.000  1438654229.000000    4.3000       20150804021029
      2  4372043.832   656334.005   -3000.000  1438726761.000000    2.3700       20150804221921
      3  4317525.747   520953.300   27690.000  1438744568.000000    2.4000       20150805031608
      4  4187827.608   249499.335   96070.000  1438844052.000000    3.0200       20150806065412
      5  4245191.012   388998.036  126440.000  1438846443.000000    3.0700       20150806073403
      6  4112039.663   219505.413   92410.000  1438868721.000000    3.3400       20150806134521
      7  4321404.819   518995.792   35030.000  1438931919.000000    2.8800       20150807071839


In [27]:
! head data/reference_mts.txt

#Number            nn            ee            dd            ne            nd            ed
# (int)          (Nm)          (Nm)          (Nm)          (Nm)           (Nm)         (Nm)
    498 -2.405400e+16  1.834400e+16  5.716000e+15  1.942300e+16  2.462000e+15 -1.199900e+16
    515 -2.897700e+16  4.746300e+16 -1.848600e+16  2.255000e+16  6.227000e+16  7.438000e+15
    877  8.420000e+14  1.569900e+16 -1.654100e+16 -7.346000e+15 -9.672000e+15  2.311000e+15
    972 -1.660000e+16  2.920000e+16 -1.260000e+16 -4.470000e+15  1.038000e+16  2.048000e+16
   2050 -1.076700e+18  1.108800e+18 -3.210000e+16  2.792000e+17  2.737000e+17  4.272000e+17
   2190 -4.960000e+14  1.781000e+15 -1.285000e+15 -8.750000e+14  5.780000e+14  1.372000e+15
   2695 -2.986000e+15 -1.063300e+16  1.361900e+16  2.492800e+16  2.414000e+15  1.749700e+16
   4181 -8.557100e+16  1.576900e+16  6.980200e+16  2.772300e+16 -2.158100e+16 -2.254200e+16
